In [2]:
import matplotlib

matplotlib.use('Agg')

import os, sys
import yaml
from argparse import ArgumentParser
from time import gmtime, strftime
from shutil import copy

from frames_dataset import FramesDataset

from modules.generator import OcclusionAwareGenerator
from modules.discriminator import MultiScaleDiscriminator
from modules.keypoint_detector import KPDetector

import torch

from train import train
from reconstruction import reconstruction
from animate import animate

In [3]:
cd D:/acads/fomm_org/

D:\acads\fomm_org


C:\Users\Acer\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
# Load the configuration file
config_path = "D:/acads/fomm_org/config/vox-256.yaml"  # Replace with your config file path
with open(config_path) as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
print("Config Loaded Successfully")


Config Loaded Successfully


In [5]:
# Initialize Generator
generator = OcclusionAwareGenerator(**config['model_params']['generator_params'], 
                                    **config['model_params']['common_params'])

# Initialize Discriminator
discriminator = MultiScaleDiscriminator(**config['model_params']['discriminator_params'], 
                                        **config['model_params']['common_params'])

# Initialize Keypoint Detector
kp_detector = KPDetector(**config['model_params']['kp_detector_params'], 
                         **config['model_params']['common_params'])

# Move models to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
generator.to(device)
discriminator.to(device)
kp_detector.to(device)

print("Models Initialized Successfully")


Models Initialized Successfully


C:\Users\Acer\AppData\Roaming\Python\Python312\site-packages\torch\functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3596.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [6]:
# Load Dataset
dataset = FramesDataset(is_train=True, **config['dataset_params'])
print("Dataset Loaded Successfully")


Use predefined train-test split.
Dataset Loaded Successfully


In [7]:
# Load checkpoint if available
checkpoint_path = None  # Replace with your checkpoint path if needed
if checkpoint_path:
    checkpoint = torch.load(checkpoint_path, map_location=device)
    generator.load_state_dict(checkpoint['generator'])
    discriminator.load_state_dict(checkpoint['discriminator'])
    kp_detector.load_state_dict(checkpoint['kp_detector'])
    print("Checkpoint Loaded Successfully")
else:
    print("No Checkpoint Provided. Starting Fresh.")


No Checkpoint Provided. Starting Fresh.


In [8]:
import cv2

# Read the image
image = cv2.imread("D:/acads/fomm_org/dp.jpg")

# Resize the image to 256x256
resized_image = cv2.resize(image, (256, 256))

# Save or display the resized image
cv2.imwrite("resized_image.jpg", resized_image)


True

In [9]:
from torchvision.transforms import ToTensor
from PIL import Image

# Test Keypoint Detection
test_image_path = "D:/acads/fomm_org/resized_image.jpg"  # Replace with a test image path

test_image = Image.open(test_image_path).convert('RGB')
test_tensor = ToTensor()(test_image).unsqueeze(0).to(device)

# Pass through Keypoint Detector
with torch.no_grad():
    keypoints = kp_detector(test_tensor)

print("Keypoints Detected Successfully")
print("Keypoints:", keypoints)


Keypoints Detected Successfully
Keypoints: {'value': tensor([[[ 0.2153,  0.8566],
         [-0.3320,  0.3262],
         [ 0.3066,  0.5359],
         [-0.1835, -0.0015],
         [-0.2097,  0.7957],
         [ 0.2020,  0.1007],
         [-0.3873,  0.5085],
         [-0.2195,  0.7425],
         [ 0.3237,  0.3248],
         [-0.2994,  0.4321]]]), 'jacobian': tensor([[[[1.0000, 0.0000],
          [0.0000, 1.0000]],

         [[1.0000, 0.0000],
          [0.0000, 1.0000]],

         [[1.0000, 0.0000],
          [0.0000, 1.0000]],

         [[1.0000, 0.0000],
          [0.0000, 1.0000]],

         [[1.0000, 0.0000],
          [0.0000, 1.0000]],

         [[1.0000, 0.0000],
          [0.0000, 1.0000]],

         [[1.0000, 0.0000],
          [0.0000, 1.0000]],

         [[1.0000, 0.0000],
          [0.0000, 1.0000]],

         [[1.0000, 0.0000],
          [0.0000, 1.0000]],

         [[1.0000, 0.0000],
          [0.0000, 1.0000]]]])}


In [10]:
# Define log directory
log_dir = "log_directory"  # Replace with your log directory path
os.makedirs(log_dir, exist_ok=True)

# Train for one epoch (or full training)
train(config, generator, discriminator, kp_detector, checkpoint_path, log_dir, dataset, device_ids=[0])


C:\Users\Acer\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Acer\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0%|          | 0/1 [00:00<?, ?it/s]